<a href="https://colab.research.google.com/github/diego-ascenciorod/Clase-Laboratorio-Estadistico/blob/main/Lab_R5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     |  Diego Ascencio Rodriguez |
| **Fecha**      |  16-02-2026 |
| **Expediente** |  755690 |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

Lo usamos porque necesitamos evaluar la capacidad de generalización del modelo, qué tan bien funciona con datos que no ha visto antes, si lo entrenamos y evaluamos con los mismos datos solo medimos el error de entrenamiento que suele ser más bajo y puede dar un falso buen desempeño debido al overfitting, al separar los datos en entrenamiento y prueba podemos estimar el error fuera de muestra y obtener una medida más realista del desempeño del modelo en situaciones reales.

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

Aunque la muestra ya representa una parte de la población no nos conviene usar todos los datos desde el inicio porque perderíamos la posibilidad de validar objetivamente el modelo, necesitamos reservar una parte para prueba para estimar cómo se comportará frente a datos nuevos, pero si usamos todo para entrenar no podemos medir el error fuera de muestra y el desempeño estimado estaría sesgado, una vez que seleccionamos y validamos el mejor modelo entonces sí podemos entrenarlo con todos los datos disponibles para su versión final.

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Motor Trend Car Road Tests (1).xlsx to Motor Trend Car Road Tests (1).xlsx


In [2]:
# importar librerías
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error

# cargar dataset
df = pd.read_excel("Motor Trend Car Road Tests (1).xlsx")

# verificar columnas
print("columnas del dataset:")
print(df.columns)

# eliminar columna model
df = df.drop(columns=["model"])

# definir variables
X = df.drop(columns=["mpg"])
y = df["mpg"]

# leave one out
loo = LeaveOneOut()
model = LinearRegression()

# entrenamiento loocv
mse_list = []

for train_index, test_index in loo.split(X):

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)

# resultados
mse_mean = np.mean(mse_list)
mse_std = np.std(mse_list)

print("numero de modelos entrenados:", len(mse_list))
print("mse promedio loocv:", round(mse_mean, 4))
print("desviacion estandar del mse:", round(mse_std, 4))



columnas del dataset:
Index(['model', 'mpg', 'cyl', 'disp', 'hp', 'drat', 'wt', 'qsec', 'vs', 'am',
       'gear', 'carb'],
      dtype='object')
numero de modelos entrenados: 32
mse promedio loocv: 12.1816
desviacion estandar del mse: 17.0674


Interpreta.

El ejercicio mide qué tan bien generaliza el modelo fuera de muestra, al ser un error cuadrático penaliza predicciones muy alejadas del valor real, aparte la desviación estándar del MSE indica qué tan variable es ese error entre las 32 iteraciones reflejando la sensibilidad del modelo a pequeños cambios en los datos de entrenamiento, si esta es relativamente alta significa que el desempeño del modelo depende bastante de qué observación se deja fuera,} lo cual es característico del LOOCV ya que reduce sesgo pero puede aumentar la varianza del estimador del error.

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [ ]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Interpreta.

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3